In [1]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point
import json

# Load the Layout Pool from Static Generation

In [2]:
with open("data/static_layout_pool.json", 'r') as openfile:
    json_object = json.load(openfile)

layouts = [[Point(p[0], p[1]) for p in lay] for lay in json_object]

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.

## Generation

- At least K = 5 trials between trials of the same pairing.
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    - Also, make sure these distances obey MIN_DIST_PATH

## Checks

- We make sure that apmprox. half the pairs happen in A->B, and the other half happens in B->A
- We want to check the number of objects within the same side across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.